In [5]:
"""
BantAI Travel-Aware Risk-Based Authentication System
Separate implementation for intelligent travel vs threat detection

Distinguishes between:
- Legitimate Filipino travelers (OFWs, tourists, business)
- Account compromise/cyber attacks

Features:
- Travel plausibility analysis
- Behavioral consistency scoring
- Impossible travel detection
- OFW-friendly risk assessment
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
from geopy.distance import geodesic
import warnings
warnings.filterwarnings('ignore')

class BantAI_TravelAware:
    """
    Travel-Aware Risk-Based Authentication for Filipino Banking
    
    Smart enough to distinguish between:
    - Juan traveling to Dubai for work (legitimate)
    - Hacker accessing from Moscow (threat)
    """
    
    def __init__(self):
        self.user_profiles = {}  # Store user behavioral baselines
        self.location_coordinates = self._load_location_data()
        self.travel_risk_zones = self._define_travel_zones()
        self.max_travel_speed_kmh = 900  # Commercial aircraft speed
        
    def _load_location_data(self):
        """
        Comprehensive geographic coordinates for travel distance calculations
        Covering major Philippine cities and global destinations
        """
        return {
            # PHILIPPINES - Major Cities and Regions
            'Manila': (14.5995, 120.9842),
            'Quezon City': (14.6760, 121.0437),
            'Makati': (14.5547, 121.0244),
            'Taguig': (14.5176, 121.0509),
            'Pasig': (14.5764, 121.0851),
            'Mandaluyong': (14.5794, 121.0359),
            'Marikina': (14.6507, 121.1029),
            'Pasay': (14.5378, 120.9896),
            'Parañaque': (14.4793, 121.0198),
            'Las Piñas': (14.4304, 120.9820),
            'Muntinlupa': (14.4037, 121.0270),
            'Caloocan': (14.6488, 120.9658),
            'Valenzuela': (14.6958, 120.9830),
            'Malabon': (14.6570, 120.9658),
            'Navotas': (14.6691, 120.9618),
            
            # Luzon Cities
            'Baguio': (16.4023, 120.5960),
            'Angeles': (15.1450, 120.5864),
            'San Fernando': (15.0332, 120.6833),
            'Dagupan': (16.0433, 120.3433),
            'Cabanatuan': (15.4891, 120.9627),
            'Olongapo': (14.8294, 120.2824),
            'Batangas': (13.7565, 121.0583),
            'Lipa': (13.9411, 121.1634),
            'Lucena': (13.9373, 121.617),
            'Naga': (13.6218, 123.1948),
            'Legazpi': (13.1391, 123.7437),
            'Iloilo City': (10.7202, 122.5621),
            'Vigan': (17.5748, 120.3871),
            'Tuguegarao': (17.6132, 121.7270),
            'Laoag': (18.1967, 120.5929),
            
            # Visayas Cities
            'Cebu City': (10.3157, 123.8854),
            'Mandaue': (10.3237, 123.9227),
            'Lapu-Lapu': (10.3103, 123.9494),
            'Bacolod': (10.6740, 122.9540),
            'Dumaguete': (9.3067, 123.3065),
            'Tacloban': (11.2447, 125.0048),
            'Ormoc': (11.0059, 124.6074),
            'Tagbilaran': (9.6496, 123.8543),
            'Roxas': (11.5854, 122.7511),
            'Kalibo': (11.7040, 122.3690),
            
            # Mindanao Cities
            'Davao City': (7.1907, 125.4553),
            'Cagayan de Oro': (8.4542, 124.6319),
            'Zamboanga': (6.9214, 122.0790),
            'Butuan': (8.9470, 125.5406),
            'Iligan': (8.2280, 124.2452),
            'Cotabato': (7.2231, 124.2467),
            'General Santos': (6.1164, 125.1716),
            'Koronadal': (6.5000, 124.8500),
            'Kidapawan': (7.0103, 125.0890),
            'Dipolog': (8.5958, 123.3417),
            'Pagadian': (7.8272, 123.4433),
            'Marawi': (8.0021, 124.2979),
            
            # MIDDLE EAST - Major OFW Destinations
            'Dubai': (25.2048, 55.2708),
            'Abu Dhabi': (24.4539, 54.3773),
            'Sharjah': (25.3573, 55.4033),
            'Ajman': (25.4052, 55.5136),
            'Al Ain': (24.2075, 55.7647),
            'Riyadh': (24.7136, 46.6753),
            'Jeddah': (21.4858, 39.1925),
            'Dammam': (26.4207, 50.0888),
            'Mecca': (21.3891, 39.8579),
            'Medina': (24.5247, 39.5692),
            'Doha': (25.2854, 51.5310),
            'Kuwait City': (29.3117, 47.4818),
            'Manama': (26.2285, 50.5860),
            'Muscat': (23.5880, 58.3829),
            'Amman': (31.9539, 35.9106),
            'Beirut': (33.8938, 35.5018),
            'Baghdad': (33.3152, 44.3661),
            'Tehran': (35.6892, 51.3890),
            'Isfahan': (32.6546, 51.6680),
            'Mashhad': (36.2605, 59.6168),
            
            # ASIA PACIFIC - Business and Tourism Hubs
            'Singapore': (1.3521, 103.8198),
            'Hong Kong': (22.3193, 114.1694),
            'Macau': (22.1987, 113.5439),
            'Tokyo': (35.6762, 139.6503),
            'Osaka': (34.6937, 135.5023),
            'Nagoya': (35.1815, 136.9066),
            'Kyoto': (35.0116, 135.7681),
            'Yokohama': (35.4437, 139.6380),
            'Seoul': (37.5665, 126.9780),
            'Busan': (35.1796, 129.0756),
            'Incheon': (37.4563, 126.7052),
            'Bangkok': (13.7563, 100.5018),
            'Phuket': (7.8804, 98.3923),
            'Pattaya': (12.9236, 100.8825),
            'Kuala Lumpur': (3.1390, 101.6869),
            'Johor Bahru': (1.4927, 103.7414),
            'Penang': (5.4164, 100.3327),
            'Jakarta': (6.2088, 106.8456),
            'Bali': (8.3405, 115.0920),
            'Surabaya': (7.2575, 112.7521),
            'Ho Chi Minh City': (10.8231, 106.6297),
            'Hanoi': (21.0285, 105.8542),
            'Da Nang': (16.0544, 108.2022),
            'Phnom Penh': (11.5564, 104.9282),
            'Vientiane': (17.9757, 102.6331),
            'Yangon': (16.8661, 96.1951),
            'Colombo': (6.9271, 79.8612),
            'Dhaka': (23.8103, 90.4125),
            'Kathmandu': (27.7172, 85.3240),
            
            # NORTH AMERICA - Filipino Diaspora Communities
            'Los Angeles': (34.0522, -118.2437),
            'San Francisco': (37.7749, -122.4194),
            'San Diego': (32.7157, -117.1611),
            'Las Vegas': (36.1699, -115.1398),
            'Phoenix': (33.4484, -112.0740),
            'Seattle': (47.6062, -122.3321),
            'Portland': (45.5152, -122.6784),
            'Sacramento': (38.5816, -121.4944),
            'Fresno': (36.7378, -119.7871),
            'San Jose': (37.3382, -121.8863),
            'New York': (40.7128, -74.0060),
            'Jersey City': (40.7178, -74.0431),
            'Philadelphia': (39.9526, -75.1652),
            'Washington DC': (38.9072, -77.0369),
            'Boston': (42.3601, -71.0589),
            'Chicago': (41.8781, -87.6298),
            'Detroit': (42.3314, -83.0458),
            'Miami': (25.7617, -80.1918),
            'Orlando': (28.5383, -81.3792),
            'Tampa': (27.9506, -82.4572),
            'Houston': (29.7604, -95.3698),
            'Dallas': (32.7767, -96.7970),
            'Austin': (30.2672, -97.7431),
            'San Antonio': (29.4241, -98.4936),
            'Denver': (39.7392, -104.9903),
            'Atlanta': (33.7490, -84.3880),
            'Honolulu': (21.3099, -157.8581),
            'Anchorage': (61.2181, -149.9003),
            
            # CANADA
            'Toronto': (43.6532, -79.3832),
            'Vancouver': (49.2827, -123.1207),
            'Montreal': (45.5017, -73.5673),
            'Calgary': (51.0447, -114.0719),
            'Edmonton': (53.5461, -113.4938),
            'Ottawa': (45.4215, -75.6972),
            'Winnipeg': (49.8951, -97.1384),
            'Quebec City': (46.8139, -71.2080),
            'Hamilton': (43.2557, -79.8711),
            
            # EUROPE - Tourism and Business
            'London': (51.5074, -0.1278),
            'Manchester': (53.4808, -2.2426),
            'Birmingham': (52.4862, -1.8904),
            'Edinburgh': (55.9533, -3.1883),
            'Glasgow': (55.8642, -4.2518),
            'Dublin': (53.3498, -6.2603),
            'Paris': (48.8566, 2.3522),
            'Lyon': (45.7640, 4.8357),
            'Marseille': (43.2965, 5.3698),
            'Rome': (41.9028, 12.4964),
            'Milan': (45.4642, 9.1900),
            'Naples': (40.8518, 14.2681),
            'Venice': (45.4408, 12.3155),
            'Madrid': (40.4168, -3.7038),
            'Barcelona': (41.3851, 2.1734),
            'Berlin': (52.5200, 13.4050),
            'Munich': (48.1351, 11.5820),
            'Frankfurt': (50.1109, 8.6821),
            'Amsterdam': (52.3676, 4.9041),
            'Brussels': (50.8503, 4.3517),
            'Vienna': (48.2082, 16.3738),
            'Zurich': (47.3769, 8.5417),
            'Geneva': (46.2044, 6.1432),
            'Stockholm': (59.3293, 18.0686),
            'Oslo': (59.9139, 10.7522),
            'Copenhagen': (55.6761, 12.5683),
            'Helsinki': (60.1699, 24.9384),
            'Warsaw': (52.2297, 21.0122),
            'Prague': (50.0755, 14.4378),
            'Budapest': (47.4979, 19.0402),
            'Bucharest': (44.4268, 26.1025),
            'Athens': (37.9838, 23.7275),
            'Istanbul': (41.0082, 28.9784),
            'Ankara': (39.9334, 32.8597),
            
            # OCEANIA
            'Sydney': (33.8688, 151.2093),
            'Melbourne': (37.8136, 144.9631),
            'Brisbane': (27.4698, 153.0251),
            'Perth': (31.9505, 115.8605),
            'Adelaide': (34.9285, 138.6007),
            'Canberra': (35.2809, 149.1300),
            'Gold Coast': (28.0167, 153.4000),
            'Newcastle': (32.9267, 151.7789),
            'Auckland': (36.8485, 174.7633),
            'Wellington': (41.2865, 174.7762),
            'Christchurch': (43.5321, 172.6362),
            
            # AFRICA
            'Lagos': (6.5244, 3.3792),
            'Abuja': (9.0765, 7.3986),
            'Kano': (12.0022, 8.5920),
            'Ibadan': (7.3775, 3.9470),
            'Cairo': (30.0444, 31.2357),
            'Alexandria': (31.2001, 29.9187),
            'Cape Town': (33.9249, 18.4241),
            'Johannesburg': (26.2041, 28.0473),
            'Durban': (29.8587, 31.0218),
            'Nairobi': (1.2921, 36.8219),
            'Addis Ababa': (9.1450, 38.7451),
            'Casablanca': (33.5731, 7.5898),
            'Tunis': (36.8065, 10.1815),
            'Algiers': (36.7538, 3.0588),
            
            # SOUTH AMERICA
            'São Paulo': (23.5558, 46.6396),
            'Rio de Janeiro': (22.9068, 43.1729),
            'Brasília': (15.8267, 47.9218),
            'Salvador': (12.9714, 38.5014),
            'Buenos Aires': (34.6118, 58.3960),
            'Córdoba': (31.4201, 64.1888),
            'Lima': (12.0464, 77.0428),
            'Bogotá': (4.7110, 74.0721),
            'Medellín': (6.2486, 75.5636),
            'Caracas': (10.4806, 66.9036),
            'Santiago': (33.4489, 70.6693),
            'Quito': (0.1807, 78.4678),
            'La Paz': (16.5000, 68.1193),
            'Montevideo': (34.9011, 56.1645),
            
            # HIGH-RISK CYBERCRIME LOCATIONS
            'Moscow': (55.7558, 37.6176),
            'St. Petersburg': (59.9311, 30.3609),
            'Novosibirsk': (55.0084, 82.9357),
            'Yekaterinburg': (56.8431, 60.6454),
            'Beijing': (39.9042, 116.4074),
            'Shanghai': (31.2304, 121.4737),
            'Shenzhen': (22.5431, 114.0579),
            'Guangzhou': (23.1291, 113.2644),
            'Hangzhou': (30.2741, 120.1551),
            'Chengdu': (30.5728, 104.0668),
            'Pyongyang': (39.0392, 125.7625),
            'Hamhung': (39.9187, 127.5358),
            'Chongjin': (41.7847, 129.7755),
            'Minsk': (53.9006, 27.5590),
            'Kiev': (50.4501, 30.5234),
            'Kharkiv': (49.9935, 36.2304),
            'Chisinau': (47.0105, 28.8638),
            'Tirana': (41.3275, 19.8187),
            'Skopje': (41.9973, 21.4280),
            'Sarajevo': (43.8486, 18.3564),
            
            # ADDITIONAL ASIAN CITIES
            'Mumbai': (19.0760, 72.8777),
            'Delhi': (28.7041, 77.1025),
            'Bangalore': (12.9716, 77.5946),
            'Chennai': (13.0827, 80.2707),
            'Hyderabad': (17.3850, 78.4867),
            'Kolkata': (22.5726, 88.3639),
            'Pune': (18.5204, 73.8567),
            'Ahmedabad': (23.0225, 72.5714),
            'Karachi': (24.8607, 67.0011),
            'Lahore': (31.5204, 74.3587),
            'Islamabad': (33.7294, 73.0931),
            'Kabul': (34.5553, 69.2075),
            'Tashkent': (41.2995, 69.2401),
            'Almaty': (43.2220, 76.8512),
            'Bishkek': (42.8746, 74.5698),
            'Dushanbe': (38.5598, 68.7870),
        }
    
    def _define_travel_zones(self):
        """
        Define comprehensive risk zones for different types of travel destinations
        Based on Filipino travel patterns, OFW destinations, and threat intelligence
        """
        return {
            'ofw_hubs': {
                'locations': [
                    # Middle East - Major OFW employment destinations
                    'Dubai', 'Abu Dhabi', 'Sharjah', 'Ajman', 'Al Ain',
                    'Riyadh', 'Jeddah', 'Dammam', 'Mecca', 'Medina',
                    'Doha', 'Kuwait City', 'Manama', 'Muscat'
                ],
                'base_risk': 0.2,  # Low risk for legitimate OFW destinations
                'description': 'Major OFW employment hubs in Middle East'
            },
            'business_hubs': {
                'locations': [
                    # Asia Pacific business centers
                    'Singapore', 'Hong Kong', 'Macau',
                    'Tokyo', 'Osaka', 'Nagoya', 'Kyoto', 'Yokohama',
                    'Seoul', 'Busan', 'Incheon',
                    'Bangkok', 'Phuket', 'Kuala Lumpur', 'Penang',
                    'Jakarta', 'Bali', 'Surabaya',
                    # Global financial centers
                    'London', 'Frankfurt', 'Zurich', 'Geneva',
                    'New York', 'Chicago', 'Boston', 'San Francisco'
                ],
                'base_risk': 0.25,
                'description': 'Regional and global business centers'
            },
            'diaspora_hubs': {
                'locations': [
                    # United States - Large Filipino communities
                    'Los Angeles', 'San Francisco', 'San Diego', 'San Jose',
                    'Las Vegas', 'Seattle', 'New York', 'Jersey City',
                    'Chicago', 'Houston', 'Miami', 'Honolulu',
                    # Canada - Filipino diaspora
                    'Toronto', 'Vancouver', 'Montreal', 'Calgary', 'Edmonton',
                    # Australia/New Zealand - Filipino communities
                    'Sydney', 'Melbourne', 'Brisbane', 'Perth', 'Adelaide',
                    'Auckland', 'Wellington'
                ],
                'base_risk': 0.3,
                'description': 'Major Filipino diaspora communities'
            },
            'tourism_destinations': {
                'locations': [
                    # Europe - Common tourist destinations
                    'Paris', 'Lyon', 'Rome', 'Milan', 'Venice', 'Naples',
                    'Madrid', 'Barcelona', 'Berlin', 'Munich',
                    'Amsterdam', 'Brussels', 'Vienna',
                    'Stockholm', 'Oslo', 'Copenhagen', 'Helsinki',
                    'Prague', 'Budapest', 'Warsaw', 'Athens',
                    'Istanbul', 'Dublin', 'Edinburgh',
                    # Other popular destinations
                    'Ho Chi Minh City', 'Hanoi', 'Da Nang',
                    'Phnom Penh', 'Vientiane', 'Yangon',
                    'Mumbai', 'Delhi', 'Bangalore'
                ],
                'base_risk': 0.35,
                'description': 'Popular tourist and cultural destinations'
            },
            'developing_markets': {
                'locations': [
                    # South/Southeast Asia
                    'Chennai', 'Hyderabad', 'Kolkata', 'Pune', 'Ahmedabad',
                    'Karachi', 'Lahore', 'Islamabad', 'Dhaka',
                    'Colombo', 'Kathmandu', 'Kabul',
                    # Central Asia
                    'Tashkent', 'Almaty', 'Bishkek', 'Dushanbe',
                    # Africa
                    'Cairo', 'Alexandria', 'Cape Town', 'Johannesburg',
                    'Nairobi', 'Addis Ababa', 'Casablanca',
                    # South America
                    'São Paulo', 'Rio de Janeiro', 'Buenos Aires', 'Lima'
                ],
                'base_risk': 0.45,
                'description': 'Developing markets with moderate risk'
            },
            'high_risk_regions': {
                'locations': [
                    # Africa - Higher risk areas
                    'Lagos', 'Abuja', 'Kano', 'Ibadan',
                    'Tunis', 'Algiers',
                    # South America - Crime hotspots
                    'Bogotá', 'Medellín', 'Caracas', 'La Paz',
                    # Conflict zones
                    'Baghdad', 'Beirut', 'Amman'
                ],
                'base_risk': 0.65,
                'description': 'Higher risk regions with security concerns'
            },
            'cybercrime_hubs': {
                'locations': [
                    # Russia - Major cybercrime source
                    'Moscow', 'St. Petersburg', 'Novosibirsk', 'Yekaterinburg',
                    # China - State-sponsored threats
                    'Beijing', 'Shanghai', 'Shenzhen', 'Guangzhou', 
                    'Hangzhou', 'Chengdu',
                    # North Korea - State actors
                    'Pyongyang', 'Hamhung', 'Chongjin',
                    # Iran - Cyber warfare
                    'Tehran', 'Isfahan', 'Mashhad',
                    # Eastern Europe - Cybercrime centers
                    'Bucharest', 'Minsk', 'Kiev', 'Kharkiv',
                    'Chisinau', 'Tirana', 'Skopje', 'Sarajevo'
                ],
                'base_risk': 0.8,
                'description': 'Known cybercrime and state-sponsored threat locations'
            },
            'philippines_domestic': {
                'locations': [
                    # Metro Manila
                    'Manila', 'Quezon City', 'Makati', 'Taguig', 'Pasig',
                    'Mandaluyong', 'Marikina', 'Pasay', 'Parañaque',
                    'Las Piñas', 'Muntinlupa', 'Caloocan', 'Valenzuela',
                    'Malabon', 'Navotas',
                    # Luzon
                    'Baguio', 'Angeles', 'San Fernando', 'Dagupan',
                    'Cabanatuan', 'Olongapo', 'Batangas', 'Lipa',
                    'Lucena', 'Naga', 'Legazpi', 'Vigan', 'Tuguegarao', 'Laoag',
                    # Visayas
                    'Cebu City', 'Mandaue', 'Lapu-Lapu', 'Iloilo City',
                    'Bacolod', 'Dumaguete', 'Tacloban', 'Ormoc',
                    'Tagbilaran', 'Roxas', 'Kalibo',
                    # Mindanao
                    'Davao City', 'Cagayan de Oro', 'Zamboanga', 'Butuan',
                    'Iligan', 'Cotabato', 'General Santos', 'Koronadal',
                    'Kidapawan', 'Dipolog', 'Pagadian', 'Marawi'
                ],
                'base_risk': 0.05,  # Very low risk for domestic access
                'description': 'Philippine domestic locations'
            }
        }
    
    def analyze_user_baseline(self, user_login_history):
        """
        Analyze user's normal behavior patterns from login history
        
        Args:
            user_login_history: List of dictionaries with login data
        """
        user_id = user_login_history[0]['user_id']
        
        # Extract behavioral patterns
        locations = [login['location'] for login in user_login_history]
        times = [datetime.strptime(login['timestamp'], '%Y-%m-%d %H:%M:%S') for login in user_login_history]
        devices = [login['device_type'] for login in user_login_history]
        countries = [login['country'] for login in user_login_history]
        
        # Calculate baseline patterns
        baseline = {
            'user_id': user_id,
            'home_locations': list(set(loc for loc, country in zip(locations, countries) if country == 'PH')),
            'common_devices': list(set(devices)),
            'typical_hours': [t.hour for t in times],
            'login_frequency': len(user_login_history),
            'countries_visited': list(set(countries)),
            'last_known_location': locations[-1],
            'last_login_time': times[-1],
            'travel_history': self._extract_travel_history(user_login_history)
        }
        
        # Store user profile
        self.user_profiles[user_id] = baseline
        
        print(f"✅ Baseline established for User {user_id}")
        print(f"   Home locations: {baseline['home_locations']}")
        print(f"   Countries visited: {baseline['countries_visited']}")
        print(f"   Common devices: {baseline['common_devices']}")
        
        return baseline
    
    def _extract_travel_history(self, login_history):
        """
        Extract travel patterns from login history
        """
        travels = []
        
        for i in range(1, len(login_history)):
            prev_login = login_history[i-1]
            curr_login = login_history[i]
            
            if prev_login['country'] != curr_login['country']:
                travel = {
                    'from_location': prev_login['location'],
                    'to_location': curr_login['location'],
                    'from_country': prev_login['country'],
                    'to_country': curr_login['country'],
                    'time_gap': (datetime.strptime(curr_login['timestamp'], '%Y-%m-%d %H:%M:%S') - 
                               datetime.strptime(prev_login['timestamp'], '%Y-%m-%d %H:%M:%S')).total_seconds() / 3600,
                    'distance_km': self._calculate_distance(prev_login['location'], curr_login['location'])
                }
                travels.append(travel)
        
        return travels
    
    def _calculate_distance(self, location1, location2):
        """
        Calculate distance between two locations
        """
        if location1 in self.location_coordinates and location2 in self.location_coordinates:
            coord1 = self.location_coordinates[location1]
            coord2 = self.location_coordinates[location2]
            return geodesic(coord1, coord2).kilometers
        return 0
    
    def _get_location_zone(self, location):
        """
        Determine which risk zone a location belongs to
        """
        for zone_name, zone_data in self.travel_risk_zones.items():
            if location in zone_data['locations']:
                return zone_name, zone_data['base_risk']
        return 'unknown', 0.5  # Default for unknown locations
    
    def analyze_travel_plausibility(self, user_id, new_login):
        """
        Analyze if travel to new location is physically plausible
        """
        if user_id not in self.user_profiles:
            return {
                'plausible': False,
                'reason': 'No user baseline established',
                'risk_modifier': 0.5
            }
        
        profile = self.user_profiles[user_id]
        last_location = profile['last_known_location']
        last_time = profile['last_login_time']
        
        new_location = new_login['location']
        new_time = datetime.strptime(new_login['timestamp'], '%Y-%m-%d %H:%M:%S')
        
        # Calculate travel requirements
        distance_km = self._calculate_distance(last_location, new_location)
        time_gap_hours = (new_time - last_time).total_seconds() / 3600
        
        if distance_km == 0:  # Same location or unknown coordinates
            return {
                'plausible': True,
                'reason': 'Same location or local area',
                'risk_modifier': 0.0,
                'distance_km': distance_km,
                'time_gap_hours': time_gap_hours
            }
        
        # Calculate minimum travel time (assuming commercial flight)
        min_travel_time_hours = distance_km / self.max_travel_speed_kmh
        buffer_time_hours = 4  # Airport procedures, layovers, etc.
        required_time_hours = min_travel_time_hours + buffer_time_hours
        
        # Check plausibility
        if time_gap_hours >= required_time_hours:
            return {
                'plausible': True,
                'reason': f'Sufficient time for travel ({time_gap_hours:.1f}h vs {required_time_hours:.1f}h required)',
                'risk_modifier': 0.0,
                'distance_km': distance_km,
                'time_gap_hours': time_gap_hours,
                'required_time_hours': required_time_hours
            }
        else:
            # Impossible travel
            return {
                'plausible': False,
                'reason': f'Impossible travel: {distance_km:.0f}km in {time_gap_hours:.1f}h (need {required_time_hours:.1f}h)',
                'risk_modifier': 0.8,  # Very high risk for impossible travel
                'distance_km': distance_km,
                'time_gap_hours': time_gap_hours,
                'required_time_hours': required_time_hours
            }
    
    def analyze_behavioral_consistency(self, user_id, new_login):
        """
        Check if user behavior remains consistent despite location change
        """
        if user_id not in self.user_profiles:
            return {'consistency_score': 0.5, 'factors': ['No baseline']}
        
        profile = self.user_profiles[user_id]
        consistency_factors = []
        consistency_score = 1.0  # Start with perfect consistency
        
        # Device consistency
        if new_login['device_type'] in profile['common_devices']:
            consistency_factors.append('Known device type')
        else:
            consistency_score -= 0.3
            consistency_factors.append('New device type')
        
        # Time pattern consistency (adjusted for timezone)
        new_hour = datetime.strptime(new_login['timestamp'], '%Y-%m-%d %H:%M:%S').hour
        if new_hour in profile['typical_hours'] or abs(new_hour - np.mean(profile['typical_hours'])) <= 3:
            consistency_factors.append('Consistent login time')
        else:
            consistency_score -= 0.2
            consistency_factors.append('Unusual login time')
        
        # Previous travel history
        if new_login['country'] in profile['countries_visited']:
            consistency_factors.append('Previously visited country')
            consistency_score += 0.1  # Bonus for familiar destinations
        else:
            consistency_factors.append('First visit to country')
        
        # Browser/session consistency (if available)
        # This would check if session tokens, browser fingerprints match
        # For demo, we'll simulate this
        if np.random.random() > 0.3:  # 70% chance of consistent browser
            consistency_factors.append('Consistent browser fingerprint')
        else:
            consistency_score -= 0.2
            consistency_factors.append('Different browser fingerprint')
        
        return {
            'consistency_score': max(0, min(1, consistency_score)),
            'factors': consistency_factors
        }
    
    def calculate_travel_aware_risk(self, user_id, new_login):
        """
        Calculate comprehensive travel-aware risk score incorporating ML predictions
        """
        # Base risk components
        risk_components = {}
        
        # 1. Location zone risk
        zone, base_location_risk = self._get_location_zone(new_login['location'])
        risk_components['location_zone'] = base_location_risk
        
        # 2. Travel plausibility
        travel_analysis = self.analyze_travel_plausibility(user_id, new_login)
        risk_components['travel_plausibility'] = travel_analysis['risk_modifier']
        
        # 3. Behavioral consistency
        behavior_analysis = self.analyze_behavioral_consistency(user_id, new_login)
        behavior_risk = 1 - behavior_analysis['consistency_score']
        risk_components['behavioral_inconsistency'] = behavior_risk
        
        # 4. Technical indicators
        technical_risk = 0.0
        technical_factors = []
        
        if new_login.get('is_attack_ip', False):
            technical_risk += 0.4
            technical_factors.append('Known attack IP')
        
        if new_login.get('high_latency', False):
            technical_risk += 0.2
            technical_factors.append('High network latency')
        
        if not new_login.get('login_successful', True):
            technical_risk += 0.3
            technical_factors.append('Failed login attempt')
        
        risk_components['technical_indicators'] = technical_risk
        
        # 5. Geographic distance factor
        if user_id in self.user_profiles:
            distance_km = self._calculate_distance(
                self.user_profiles[user_id]['last_known_location'],
                new_login['location']
            )
            # Risk increases with distance, but caps at 0.3
            distance_risk = min(0.3, distance_km / 10000)  # 10,000km = max distance risk
            risk_components['distance'] = distance_risk
        else:
            risk_components['distance'] = 0.2
        
        # 6. ML Model prediction (if available)
        ml_risk = 0.5  # Default if no model
        if hasattr(self, 'model') and self.model is not None:
            # Get the last known login from user profile
            if user_id in self.user_profiles:
                last_login = {
                    'time': self.user_profiles[user_id]['last_login_time'],
                    'Country': self.user_profiles[user_id]['last_known_location'].split(',')[-1].strip(),
                    'City': self.user_profiles[user_id]['last_known_location'].split(',')[0].strip(),
                    'Device Type': new_login.get('device_type', 'unknown'),
                    'Round-Trip Time [ms]': 0,  # Default value
                    'Is Attack IP': new_login.get('is_attack_ip', False),
                    'Login Successful': new_login.get('login_successful', True),
                    'User Agent String': '',  # Default value
                    'ASN': None  # Default value
                }
                try:
                    ml_risk = self.ml_risk_for_pair(last_login, new_login)
                except Exception as e:
                    print(f"Note: ML prediction failed - {str(e)}")
        
        risk_components['ml_prediction'] = ml_risk
        
        # Calculate weighted final risk score with ML
        weights = {
            'location_zone': 0.2,
            'travel_plausibility': 0.2,
            'behavioral_inconsistency': 0.15,
            'technical_indicators': 0.15,
            'distance': 0.1,
            'ml_prediction': 0.2  # Give ML significant weight
        }
        
        final_risk = sum(risk_components[component] * weights[component] 
                        for component in risk_components)
        
        # Cap at 1.0
        final_risk = min(1.0, final_risk)
        
        return {
            'final_risk_score': final_risk,
            'risk_components': risk_components,
            'travel_analysis': travel_analysis,
            'behavior_analysis': behavior_analysis,
            'location_zone': zone,
            'technical_factors': technical_factors
        }
    
    def generate_travel_aware_explanation(self, risk_analysis, language='english'):
        """
        Generate human-readable explanation for the risk decision
        """
        risk_score = risk_analysis['final_risk_score']
        travel_info = risk_analysis['travel_analysis']
        behavior_info = risk_analysis['behavior_analysis']
        
        # Determine risk level
        if risk_score < 0.3:
            risk_level = "LOW"
            action = "ALLOW"
            message_key = "low_risk"
        elif risk_score < 0.6:
            risk_level = "MEDIUM"
            action = "ALLOW_WITH_OTP"
            message_key = "medium_risk"
        else:
            risk_level = "HIGH"
            action = "BLOCK" if not travel_info['plausible'] else "STRICT_VERIFICATION"
            message_key = "high_risk"
        
        # Build explanation factors
        explanation_factors = []
        
        # Travel plausibility
        if travel_info['plausible']:
            explanation_factors.append(f"✅ Travel is plausible ({travel_info['reason']})")
        else:
            explanation_factors.append(f"❌ {travel_info['reason']}")
        
        # Behavioral consistency
        consistency_pct = behavior_info['consistency_score'] * 100
        explanation_factors.append(f"🔍 Behavior consistency: {consistency_pct:.0f}%")
        
        # Location zone
        zone = risk_analysis['location_zone']
        zone_info = self.travel_risk_zones.get(zone, {})
        explanation_factors.append(f"📍 Location: {zone_info.get('description', 'Unknown zone')}")
        
        # Technical factors
        if risk_analysis['technical_factors']:
            explanation_factors.extend([f"⚠️ {factor}" for factor in risk_analysis['technical_factors']])
        
        return {
            'risk_score': risk_score,
            'risk_level': risk_level,
            'action': action,
            'explanation_factors': explanation_factors,
            'travel_plausible': travel_info['plausible'],
            'behavior_consistent': behavior_info['consistency_score'] > 0.7,
            'recommendation': self._get_recommendation(risk_score, travel_info, behavior_info)
        }
    
    def _get_recommendation(self, risk_score, travel_info, behavior_info):
        """
        Generate specific recommendations based on analysis
        """
        if not travel_info['plausible']:
            return "BLOCK: Impossible travel detected. Manual review required."
        
        if risk_score < 0.3 and behavior_info['consistency_score'] > 0.8:
            return "ALLOW: Legitimate travel with consistent behavior."
        
        if risk_score < 0.6 and travel_info['plausible']:
            return "ALLOW with SMS OTP: Possible legitimate travel, verify with additional authentication."
        
        return "STRICT VERIFICATION: High-risk login requiring manual review and multiple authentication factors."

def demo_travel_scenarios():
    """
    Demonstrate travel-aware RBA with realistic scenarios
    """
    print("🛫 BANTAI TRAVEL-AWARE RBA DEMO")
    print("="*50)
    
    # Initialize the travel-aware system
    bantai_travel = BantAI_TravelAware()
    
    # Sample user: Juan, an OFW from Manila
    user_id = "juan_dela_cruz_123"
    
    # Juan's login history (establishing baseline)
    juan_history = [
        {
            'user_id': user_id,
            'timestamp': '2024-01-01 09:00:00',
            'location': 'Manila',
            'country': 'PH',
            'device_type': 'mobile',
            'login_successful': True
        },
        {
            'user_id': user_id,
            'timestamp': '2024-01-03 14:30:00',
            'location': 'Makati',
            'country': 'PH',
            'device_type': 'mobile',
            'login_successful': True
        },
        {
            'user_id': user_id,
            'timestamp': '2024-01-05 11:15:00',
            'location': 'Quezon City',
            'country': 'PH',
            'device_type': 'desktop',
            'login_successful': True
        },
        {
            'user_id': user_id,
            'timestamp': '2024-01-08 16:45:00',
            'location': 'Manila',
            'country': 'PH',
            'device_type': 'mobile',
            'login_successful': True
        },
        {
            'user_id': user_id,
            'timestamp': '2024-01-10 08:20:00',
            'location': 'Manila',
            'country': 'PH',
            'device_type': 'mobile',
            'login_successful': True
        }
    ]
    
    # Establish baseline
    print("\n📊 ESTABLISHING USER BASELINE...")
    baseline = bantai_travel.analyze_user_baseline(juan_history)
    
    # Test scenarios
    test_scenarios = [
        {
            'name': '✈️ Legitimate OFW Travel to Dubai',
            'login': {
                'user_id': user_id,
                'timestamp': '2024-01-20 19:30:00',  # 10 days later, enough time to travel
                'location': 'Dubai',
                'country': 'AE',
                'device_type': 'mobile',  # Same device
                'login_successful': True,
                'is_attack_ip': False,
                'high_latency': False
            }
        },
        {
            'name': '🚨 Impossible Travel Attack',
            'login': {
                'user_id': user_id,
                'timestamp': '2024-01-20 20:30:00',  # 1 hour after Dubai login
                'location': 'Moscow',
                'country': 'RU',
                'device_type': 'desktop',  # Different device
                'login_successful': False,
                'is_attack_ip': True,
                'high_latency': True
            }
        },
        {
            'name': '🛂 Business Trip to Singapore',
            'login': {
                'user_id': user_id,
                'timestamp': '2024-01-25 10:00:00',  # 5 days later from Dubai
                'location': 'Singapore',
                'country': 'SG',
                'device_type': 'mobile',  # Same device
                'login_successful': True,
                'is_attack_ip': False,
                'high_latency': False
            }
        },
        {
            'name': '👨‍💼 Return to Philippines',
            'login': {
                'user_id': user_id,
                'timestamp': '2024-02-01 15:00:00',  # 1 week later
                'location': 'Manila',
                'country': 'PH',
                'device_type': 'mobile',
                'login_successful': True,
                'is_attack_ip': False,
                'high_latency': False
            }
        }
    ]
    
    # Analyze each scenario
    print(f"\n🔍 ANALYZING TRAVEL SCENARIOS...")
    print("="*50)
    
    for scenario in test_scenarios:
        print(f"\n{scenario['name']}")
        print("-" * 40)
        
        # Calculate risk
        risk_analysis = bantai_travel.calculate_travel_aware_risk(
            user_id, scenario['login']
        )
        
        # Generate explanation
        explanation = bantai_travel.generate_travel_aware_explanation(risk_analysis)
        
        # Display results
        print(f"📍 Location: {scenario['login']['location']}, {scenario['login']['country']}")
        print(f"🎯 Risk Score: {explanation['risk_score']:.2f} ({explanation['risk_level']})")
        print(f"⚡ Action: {explanation['action']}")
        print(f"✅ Travel Plausible: {explanation['travel_plausible']}")
        print(f"🔄 Behavior Consistent: {explanation['behavior_consistent']}")
        print(f"💡 Recommendation: {explanation['recommendation']}")
        
        print("\n🔍 Detailed Analysis:")
        for factor in explanation['explanation_factors']:
            print(f"   {factor}")
    
    return bantai_travel

# Usage example
if __name__ == "__main__":
    # Run the travel-aware demo
    travel_system = demo_travel_scenarios()
    
    print("\n💡 Usage for Custom Scenarios:")
    print("# 1. Establish user baseline:")
    print("baseline = travel_system.analyze_user_baseline(user_login_history)")
    print("\n# 2. Analyze new login:")
    print("risk_analysis = travel_system.calculate_travel_aware_risk(user_id, new_login)")
    print("explanation = travel_system.generate_travel_aware_explanation(risk_analysis)")

🛫 BANTAI TRAVEL-AWARE RBA DEMO

📊 ESTABLISHING USER BASELINE...
✅ Baseline established for User juan_dela_cruz_123
   Home locations: ['Quezon City', 'Makati', 'Manila']
   Countries visited: ['PH']
   Common devices: ['mobile', 'desktop']

🔍 ANALYZING TRAVEL SCENARIOS...

✈️ Legitimate OFW Travel to Dubai
----------------------------------------
📍 Location: Dubai, AE
🎯 Risk Score: 0.20 (LOW)
⚡ Action: ALLOW
✅ Travel Plausible: True
🔄 Behavior Consistent: True
💡 Recommendation: ALLOW with SMS OTP: Possible legitimate travel, verify with additional authentication.

🔍 Detailed Analysis:
   ✅ Travel is plausible (Sufficient time for travel (251.2h vs 11.7h required))
   🔍 Behavior consistency: 80%
   📍 Location: Major OFW employment hubs in Middle East

🚨 Impossible Travel Attack
----------------------------------------
📍 Location: Moscow, RU
🎯 Risk Score: 0.46 (MEDIUM)
⚡ Action: ALLOW_WITH_OTP
✅ Travel Plausible: True
🔄 Behavior Consistent: True
💡 Recommendation: ALLOW with SMS OTP: Poss

In [7]:
# Load the trained model from main1_final.ipynb
bantai_final = BantAI_TravelAware(
    cache_file="geocache.json",  # This is the correct parameter name
    ml_model_path="bantai_model.pkl",
    geocode_delay=1.0  # Added required parameter
)

try:
    # Load the pre-trained model
    bantai_final.load_model()
    print("✅ Successfully loaded pre-trained model from main1_final.ipynb")
    
    # Now bantai_final has both the travel-aware features and ML capabilities
    print("🎯 System ready for unified risk assessment with:")
    print("   - Travel plausibility analysis")
    print("   - Machine learning predictions")
    print("   - Behavioral consistency scoring")
    print("   - Technical indicators analysis")
except FileNotFoundError:
    print("⚠️ Pre-trained model not found. Please run main1_final.ipynb first to train the model.")

TypeError: BantAI_TravelAware.__init__() got an unexpected keyword argument 'cache_file'